In [25]:
import cv2
import numpy as np

def tile_frame(frame, tile_size=320, overlap=0.4):
    stride = int(tile_size * (1 - overlap))
    tiles = []
    tile_positions = []  # To track the original position of each tile in the frame
    
    for y in range(0, frame.shape[0] - tile_size + 1, stride):
        for x in range(0, frame.shape[1] - tile_size + 1, stride):
            tile = frame[y:y+tile_size, x:x+tile_size]
            tiles.append(tile)
            tile_positions.append((x, y))
            
    return tiles, tile_positions

In [22]:
from ultralytics import YOLO

# Load the YOLO model
model = YOLO('/Users/apaula/Library/CloudStorage/GoogleDrive-elysiacristata@gmail.com/My Drive/datasets/runs/pose/train9/weights/best.pt')


In [18]:
def detect_pose_in_tile(tile, model, confidence_threshold=0.3):
    results = model(tile)
    detections = []

    # Accessing the first (and presumably only) Results object
    results_object = results[0]

    # Extracting bounding boxes and their confidence scores from 'data' attribute
    if hasattr(results_object, 'boxes') and results_object.boxes is not None:
        for i, box_data in enumerate(results_object.boxes.data):
            # box_data format: [x_min, y_min, x_max, y_max, conf, cls]
            conf = box_data[4].item()  # Convert tensor to Python scalar
            if conf > confidence_threshold:
                detection = {
                    'bbox': box_data[:4].cpu().numpy(),  # Convert bbox tensor to numpy array
                    'confidence': conf,
                    'class': box_data[5].item()  # Class of the detection
                }
                # Add keypoints if available
                if hasattr(results_object, 'keypoints') and results_object.keypoints is not None and len(results_object.keypoints.data) > i:
                    kp_data = results_object.keypoints.data[i]
                    detection['keypoints'] = {
                        'points': kp_data[:, :2].cpu().numpy(),  # Keypoint coordinates
                        'confidence': kp_data[:, 2].cpu().numpy()  # Keypoint confidences
                    }
                detections.append(detection)

    return detections


In [13]:
def visualize_detections_on_tile(tile, detections):
    for detection in detections:
        # Draw bounding box
        cv2.rectangle(tile, (int(detection['bbox'][0]), int(detection['bbox'][1])), 
                      (int(detection['bbox'][2]), int(detection['bbox'][3])), (0, 255, 0), 2)
        # For keypoints visualization, adjust based on your keypoints data structure
        # e.g., cv2.drawKeypoints(tile, keypoints, ...)
    
    cv2.imshow('Detection', tile)
    cv2.waitKey(0)

In [14]:
import os
import cv2

def draw_detections_and_save(tile, detections, save_dir, tile_index):
    """
    Draw detections on a tile and save the result to disk.
    
    Parameters:
    - tile: The image tile to draw detections on.
    - detections: A list of detections where each detection is a dictionary
      containing 'bbox' (bounding box as [xmin, ymin, xmax, ymax]) and 'confidence'.
    - save_dir: Directory to save the output images.
    - tile_index: Index of the tile to create a unique filename.
    """
    # Ensure the save directory exists
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    
    # Draw detections on the tile
    for detection in detections:
        bbox = detection['bbox']
        cv2.rectangle(tile, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])), (0, 255, 0), 2)
        confidence = detection.get('confidence', 0)
        cv2.putText(tile, f'{confidence:.2f}', (int(bbox[0]), int(bbox[1]-5)), cv2.FONT_HERSHEY_SIMPLEX, 
                    0.5, (255, 255, 255), 2)
    
    # Save the tile with detections drawn on it
    save_path = os.path.join(save_dir, f'tile_{tile_index}.jpg')
    cv2.imwrite(save_path, tile)


In [15]:
def process_tiles_and_save_detections(tiles, model, save_dir='detected_tiles'):
    """
    Process each tile for pose detection, draw the detections, and save the results.
    
    Parameters:
    - tiles: List of image tiles to process.
    - model: The loaded pose detection model.
    - save_dir: Directory to save the output images with detections.
    """
    for i, tile in enumerate(tiles):
        # Assuming your detection function returns a list of detections for the tile
        detections = detect_pose_in_tile(tile, model)
        draw_detections_and_save(tile, detections, save_dir, i)

In [26]:
# Example call to process tiles and save detections
    # Load the image
image_path = '/Volumes/T7-August/sample_frame.jpg'
frame = cv2.imread(image_path)
tiles, tile_positions = tile_frame(frame)  # Unpack both tiles and their positions
model = YOLO('/Users/apaula/Library/CloudStorage/GoogleDrive-elysiacristata@gmail.com/My Drive/datasets/runs/pose/train9/weights/best.pt')  # Load your model
process_tiles_and_save_detections(tiles, model)



0: 320x320 (no detections), 125.2ms
Speed: 1.5ms preprocess, 125.2ms inference, 0.3ms postprocess per image at shape (1, 3, 320, 320)

0: 320x320 (no detections), 124.5ms
Speed: 0.5ms preprocess, 124.5ms inference, 0.3ms postprocess per image at shape (1, 3, 320, 320)

0: 320x320 (no detections), 122.1ms
Speed: 0.6ms preprocess, 122.1ms inference, 0.3ms postprocess per image at shape (1, 3, 320, 320)

0: 320x320 (no detections), 116.9ms
Speed: 0.6ms preprocess, 116.9ms inference, 0.3ms postprocess per image at shape (1, 3, 320, 320)

0: 320x320 5 fishs, 117.9ms
Speed: 0.5ms preprocess, 117.9ms inference, 0.6ms postprocess per image at shape (1, 3, 320, 320)

0: 320x320 18 fishs, 117.2ms
Speed: 0.4ms preprocess, 117.2ms inference, 0.6ms postprocess per image at shape (1, 3, 320, 320)

0: 320x320 22 fishs, 120.8ms
Speed: 0.5ms preprocess, 120.8ms inference, 0.5ms postprocess per image at shape (1, 3, 320, 320)

0: 320x320 19 fishs, 130.8ms
Speed: 0.5ms preprocess, 130.8ms inference, 0.6